In [1]:
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import csv
import operator
import Geohash
from build import TreeBuilding
from decision import haversine
from sklearn.cross_validation import train_test_split
import pickle

%matplotlib inline

In [2]:
air_trajs = pd.read_csv('air_trajs.csv',
                        #index_col=,
                        usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [3]:
trajectories = list(air_trajs['POLYLINE'])
targets = list(air_trajs['Destination'])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(trajectories, targets, train_size=0.70, random_state = 0)

In [5]:
tb = TreeBuilding()
tb = tb.fit(X_train, y_train, 1, 100)

length 3029
('len', 3029)


KeyboardInterrupt: 

In [5]:
tb.root.prediction

[-8.6112507372920515, 41.18673345887246]

In [31]:
def predict_batch(traj_tree, trajectories):
    return [predict_one(traj_tree, trajectory) for trajectory in trajectories] 
    
def predict_one(traj_tree, trajectory):
    current_node = traj_tree.root
    
    for idx, point in enumerate(trajectory):
        if len(trajectory) > (idx + 1):
            if (current_node.left == None) and (current_node.right == None):
                prediction = current_node.prediction
                
            elif (current_node.left == None) and (current_node.right != None):
                current_node = current_node.right
                
            elif (current_node.left != None) and (current_node.right == None):
                current_node = current_node.left
                
            else:
                if haversine(point, current_node.decision_point) < current_node.radius:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
                    
        else:
            prediction = current_node.prediction
            
    #return [float(prediction[0]), float(prediction[1])]
    return prediction

In [33]:
def prediction_error(y_val, y_hat):
    return np.mean(np.power([haversine(y_val[i], y_hat[i]) for i in xrange(len(y_val))], 2))

In [69]:
y_hat = predict_batch(tb, X_test)

In [70]:
prediction_error(y_test, y_hat)

523.71657286206573

In [43]:
def mean_points_alpha(points):
    lats = [point[0] for point in points]
    lons = [point[1] for point in points]
    return [np.mean(lats), np.mean(lons)]

def sqare_error_alpha(points):
    pmean = mean_points_alpha(points)
    #print np.mean(np.power([haversine(point, pmean) for point in points], 2))
    return np.mean(np.power([haversine(point, pmean) for point in points], 2))

In [65]:
sqare_error_alpha(y_test)

492.11715109521219

In [50]:
print tb.root.decision_point

[ -8.666676  41.233914]


In [71]:
pickle.dump(tb, open( "tree_100.p", "wb" ))

In [58]:
load_tree = pickle.load( open( "tree_20.p", "rb" ) )

In [59]:
y_hat = predict_batch(load_tree, X_test)
prediction_error(y_test, y_hat)

536.48747393301142